In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder,MinMaxScaler,LabelEncoder
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

In [39]:
df=pd.read_csv('/kaggle/input/nigeria-crime/Nigeria_1997-2024_Sep20.csv')
df.head()

,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,tags,timestamp
0,NIG38575,2024-09-20,2024,1,Demonstrations,Protests,Peaceful protest,Protesters (Nigeria),Refugees/IDPs (Nigeria),6,...,Bolori,11.8826,13.0890,1,Whatsapp,New media,"On 20 September 2024, IDPs (flood victims) fro...",0,crowd size=no report,1727134598
1,NIG38585,2024-09-20,2024,2,Demonstrations,Protests,Peaceful protest,Protesters (Nigeria),Muslim Group (Nigeria),6,...,Ile-Ife,7.4824,4.5603,1,Daily Trust (Nigeria),National,"Around 20 September 2024 (as reported), hundre...",0,crowd size=hundreds,1727134598
2,NIG38581,2024-09-19,2024,1,Demonstrations,Protests,Peaceful protest,Protesters (Nigeria),NaN,6,...,Agodi,7.4035,3.9132,1,Daily Post (Nigeria); Guardian (Nigeria); Saha...,National-Regional,"On 19 September 2024, for a second consecutive...",0,crowd size=hundreds,1727134598
3,NIG38588,2024-09-19,2024,1,Strategic developments,Strategic developments,Disrupted weapons use,Police Forces of Nigeria (2023-),NaN,1,...,Buruku,10.6179,7.2331,1,Daily Post (Nigeria); Nigeria Punch,National,"Weapons seizure: On 19 September 2024, Police ...",0,NaN,1727134598
4,NIG38591,2024-09-19,2024,1,Demonstrations,Protests,Peaceful protest,Protesters (Nigeria),PDP: People's Democratic Party,6,...,Wanno,7.1389,6.5724,1,Vanguard (Nigeria),National,"On 19 September 2024, PDP youths protested at ...",0,crowd size=no report,1727134598


In [40]:
for l in df.columns:
    print(l,"=",df[l].unique().shape)

event_id_cnty = (38076,)
event_date = (6513,)
year = (28,)
time_precision = (3,)
disorder_type = (4,)
event_type = (6,)
sub_event_type = (24,)
actor1 = (881,)
assoc_actor_1 = (1219,)
inter1 = (8,)
actor2 = (884,)
assoc_actor_2 = (1382,)
inter2 = (9,)
interaction = (43,)
civilian_targeting = (2,)
iso = (2,)
region = (1,)
country = (1,)
admin1 = (38,)
admin2 = (752,)
admin3 = (1,)
location = (5116,)
latitude = (4935,)
longitude = (5013,)
geo_precision = (3,)
source = (2824,)
source_scale = (23,)
notes = (36356,)
fatalities = (134,)
tags = (222,)
timestamp = (2107,)


Dropping unnecessary columns

In [41]:
df.drop(columns=['event_date', 'year','timestamp','admin3','country','region','iso','event_id_cnty','admin1','admin2','latitude','longitude','assoc_actor_1','assoc_actor_2','tags'], axis=1, inplace=True)

Label encoding the categorical columns


In [42]:
cat = df.select_dtypes(exclude=np.number).columns.tolist()
encoders={}

for col in cat:
    if col=='notes': 
        continue
    l = LabelEncoder()
    df[col] = l.fit_transform(df[col])
    encoders[col] = l

Dropping NULL values

In [43]:
df.isna().sum()

time_precision        0
disorder_type         0
event_type            0
sub_event_type        0
actor1                0
inter1                0
actor2                0
inter2                0
interaction           0
civilian_targeting    0
location              0
geo_precision         0
source                0
source_scale          0
notes                 0
fatalities            0
dtype: int64

In [23]:
df.dropna(inplace=True)

Fine-tuning DistilBert on the given dataset

In [50]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=512)

tokenized_texts = df['notes'].apply(tokenize_function).tolist()
input_ids = torch.tensor([text['input_ids'] for text in tokenized_texts])
attention_masks = torch.tensor([text['attention_mask'] for text in tokenized_texts])
labels = torch.tensor(df['fatalities'].values, dtype=torch.float)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx]
        }
    
train_input_ids, val_input_ids, train_attention_masks, val_attention_masks, train_labels, val_labels = train_test_split(input_ids, attention_masks, labels, test_size=0.2, random_state=42)
train = CustomDataset(train_input_ids, train_attention_masks, train_labels)
val = CustomDataset(val_input_ids, val_attention_masks, val_labels)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

args = TrainingArguments(
    output_dir='./results',  
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    save_strategy='epoch',  
    save_total_limit=2,  
    load_best_model_at_end=True,  
    metric_for_best_model='eval_loss',  
    logging_dir='./logs',
    logging_steps=10,
    weight_decay=0.01,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train,
    eval_dataset=val,
)
trainer.train()
results = trainer.evaluate()
print(results)
trainer.save_model('./best_model')  


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimen

Epoch,Training Loss,Validation Loss
1,418.768800,85.201973
2,11.282000,74.619965
3,1.024800,64.019493
4,35.794800,62.385193
5,51.418500,60.507668


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

{'eval_loss': 60.507667541503906, 'eval_runtime': 68.4906, 'eval_samples_per_second': 111.198, 'eval_steps_per_second': 6.95, 'epoch': 5.0}


predicting values from bert and adding it to the dataframe

In [52]:
model.eval()  
def process_text(text):
    inp = tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    inp = {key: value.to(device) for key, value in inp.items()}
    with torch.no_grad():
        out = model(**inp)
    logits = out.logits
    pred = logits.item()  
    return pred
df['bert_predictions'] = df['notes'].apply(process_text)

In [55]:
df.drop(columns=['notes'], axis=1, inplace=True)

Scaling the columns with standard scaler


In [87]:
scaler = StandardScaler()
df1 = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

Processing the data for training by splitting into train/test set

In [88]:
X=df1.drop(columns=['fatalities'],axis=1)
y=df1['fatalities']
X1=X.copy()

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Creating stacked models manually by passing BERT predictions as an input feature to traditional regression models


In [105]:
lg=LinearRegression()
lg.fit(X_train,y_train)
y_pred=lg.predict(X_test)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"R2 Score: {r2}")
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Absolute Percentage Error: {mape}")


R2 Score: 0.5671837324465934
Mean Squared Error: 0.2876493651149577
Mean Absolute Error: 0.10188835585084487
Mean Absolute Percentage Error: 0.6744110464146141


In [101]:
rf=RandomForestRegressor()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"R2 Score: {r2}")
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Absolute Percentage Error: {mape}")


R2 Score: 0.6447704364071924
Mean Squared Error: 0.23608530015551235
Mean Absolute Error: 0.04533654829951317
Mean Absolute Percentage Error: 0.2631073273443123


In [106]:
gb=GradientBoostingRegressor()
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"R2 Score: {r2}")
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Absolute Percentage Error: {mape}")

R2 Score: 0.6537413475551624
Mean Squared Error: 0.23012323936975912
Mean Absolute Error: 0.045944434584203345
Mean Absolute Percentage Error: 0.26966939038252363


Feature importance based on the Random forest model


In [60]:

feature_importances = rf.feature_importances_
feature_names = X1.columns

# Create a DataFrame for better visualization
importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importances_df = importances_df.sort_values('Importance', ascending=False)

print(importances_df)


               Feature  Importance
14    bert_predictions    0.775749
10            location    0.047179
4               actor1    0.033807
12              source    0.033561
0       time_precision    0.022135
8          interaction    0.022027
3       sub_event_type    0.020830
2           event_type    0.014782
13        source_scale    0.008523
6               actor2    0.005226
5               inter1    0.004887
11       geo_precision    0.004672
7               inter2    0.003642
9   civilian_targeting    0.002924
1        disorder_type    0.000059
